In [ ]:
#Set google drive
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:

#Install miniconda and rdkit
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2019-11-13 04:46:22--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71785000 (68M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  68.46M   243MB/s    in 0.3s    

2019-11-13 04:46:23 (243 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [71785000/71785000]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.2.0=py37_0
    - ca-certificates==2019.10.16=0
    - certifi==2019.9.11=py37_0
    - cffi==1.13.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_

In [ ]:
#Read smiles files
from rdkit import Chem
max_natoms = 50
with open('/content/gdrive/My Drive/Colab Notebooks/smiles.txt') as f:
  smiles = f.readlines()[:]
  smiles = [s.strip() for s in smiles]
  smiles = [s.split()[1] for s in smiles]
  smiles = [s for s in smiles[:30000] if Chem.MolFromSmiles(s).GetNumAtoms()<max_natoms]

print ('Number of smiles:', len(smiles))

Number of smiles: 29993


In [ ]:
#Calculate  LogP of each molecule
from rdkit import Chem
from rdkit.Chem.Crippen import MolLogP
import numpy as np
import torch
import time

Y = []
num_data = 20000
st = time.time()
for s in smiles[:num_data]:
  m = Chem.MolFromSmiles(s)
  logp = MolLogP(m)
  Y.append(logp)
  #Y.append(1)
end = time.time()

print (f'Time:{(end-st):.3f}')

Time:10.668


In [ ]:
#Dataset
from torch.utils.data import Dataset, DataLoader
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
class MolDataset(Dataset):
    def __init__(self, smiles, properties, max_natoms, norm = 0):
      self.smiles = smiles
      self.properties = properties
      self.max_natoms = max_natoms
      self.norm = norm
      
    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        #blank
        s = self.smiles[idx]
        m = Chem.MolFromSmiles(s)
        natoms = m.GetNumAtoms()

        #generate adjacency matrix
        A = GetAdjacencyMatrix(m) + np.eye(natoms)

        #normalizing adjacency matrix
        row_sums = A.sum(0)
        row_sums = np.sqrt(row_sums)
        if (self.norm == 1):
          A = A / row_sums[:, np.newaxis]

        if (self.norm == 2):
          A = A / row_sums[:, np.newaxis]
          A = A / row_sums[np.newaxis, :]

        A_padding = np.zeros((self.max_natoms, self.max_natoms))
        A_padding[:natoms, :natoms] = A

        X = [self.atom_feature(m,i) for i in range(natoms)]
        for i in range(natoms, max_natoms):
          X.append(np.zeros(28))
        X = np.array(X)

        sample = dict()
        sample['X'] = torch.from_numpy(X)
        sample['A'] = torch.from_numpy(A_padding)
        sample['Y'] = self.properties[idx]

        # sample['X'] = X
        # sample['A'] = A
        # sample['Y'] = self.properties[idx]
        # sample['L'] = natoms

        return sample

    def one_of_k_encoding(self, x, allowable_set):
        if x not in allowable_set:
            raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
        #print list((map(lambda s: x == s, allowable_set)))
        return list(map(lambda s: x == s, allowable_set))

    def one_of_k_encoding_unk(self, x, allowable_set):
        """Maps inputs not in the allowable set to the last element."""
        if x not in allowable_set:
            x = allowable_set[-1]
        return list(map(lambda s: x == s, allowable_set))

    def atom_feature(self, m, atom_i):

        atom = m.GetAtomWithIdx(atom_i)
        return np.array(self.one_of_k_encoding_unk(atom.GetSymbol(),
                                          ['C', 'N', 'O', 'S', 'F', 'P', 'Cl', 'Br', 'B', 'H']) +
                        self.one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5]) +
                        self.one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4]) +
                        self.one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5]) +
                        [atom.GetIsAromatic()])    # (10, 6, 5, 6, 1) --> total 28

    

In [ ]:
#Collate fn
def my_collate(batch):
    sample = dict()
    L = torch.Tensor([b['L'] for b in batch])

    maxnum = int(L.max())

    # A_padding = np.zeros((maxnum, maxnum))
    # X_padding = np.zeros((maxnum, 28))
    # A = torch.Tensor([np.pad(b['A'], ((0, int(maxnum - b['L'])), (0, int(maxnum - b['L']))), 'constant', constant_values= 0) for b in batch])
    # X = torch.Tensor([np.pad(b['X'], ((0, int(maxnum - b['L'])), (0, 0)), 'constant', constant_values= 0) for b in batch])

    Y_list =[]
    A_list = []
    X_list = []


    for b in batch:
      A_tmp = np.zeros((maxnum, maxnum))
      X_tmp = np.zeros((maxnum, 28))

      A_tmp[:int(b['L']), :int(b['L'])] = b['A']
      X_tmp[:int(b['L']), :] = b['X']

      A_list.append(A_tmp)
      X_list.append(X_tmp)
      Y_list.append(b['Y'])
    

    sample['X'] = torch.Tensor(X_list)
    sample['A'] = torch.Tensor(A_list)
    sample['Y'] = torch.Tensor(Y_list)
    

    return sample

In [ ]:
#Model
import torch
import torch.nn as nn
import torch.nn.functional as F

class GConvRegressor(torch.nn.Module):
  def __init__(self, n_feature=128, n_layer = 10, skip = False, n_skip = 3, rnn_mode = False, n_rnn_layer = 1, dropout = 0):
    super(GConvRegressor, self).__init__()
    #blank
    self.W = nn.ModuleList([nn.Linear(n_feature, n_feature) for i in range(n_layer)])
    self.embedding = nn.Linear(28, n_feature)
    if(rnn_mode):
      self.rnn = nn.GRU(input_size=max_natoms * n_feature, hidden_size=max_natoms * n_feature, num_layers=n_rnn_layer, dropout = dropout)
      self.fc = nn.Linear(n_feature * max_natoms * n_layer, 1)
    else:
      self.fc = nn.Linear(n_feature, 1)
    self.n_skip = n_skip
    self.skip = skip
    self.n_feature = n_feature
    self.rnn_mode = rnn_mode

  def forward(self, x, A):
    #blank
    x = self.embedding(x)
    before_x = x
    torchs = []

    if(self.rnn_mode):
      for i in range(len(self.W)):
        x = self.W[i](x)
        x = torch.einsum('ijk,ikl->ijl', [A.clone(), x])
        if(self.skip):
          if i%self.n_skip == 1:
            x = F.relu(x + before_x)
            before_x = x
          else:
            x = F.relu(x)
        else:
          x = F.relu(x)
        
        torchs.append(x)
      
      torchs = torch.stack(torchs)
      #print(torchs.size())
      torchs = torchs.reshape(torchs.size(0), torchs.size(1), -1)
      output, h_n = self.rnn(torchs)

      output = output.permute((1,0,2))
      output = output.reshape(output.size(0), -1)
      #print(output.size())
      retval = self.fc(output)

      return retval

    else:
      for i in range(len(self.W)):
        x = self.W[i](x)
        x = torch.einsum('ijk,ikl->ijl', [A.clone(), x])
        if(self.skip):
          if i%self.n_skip == 1:
            x = F.relu(x + before_x)
            before_x = x
          else:
            x = F.relu(x)
        else:
          x = F.relu(x)
      
      x = x.mean(1)

      retval = self.fc(x)

      return retval


In [ ]:
#Train model
import time
lr = 1e-4
# model = GConvRegressor(128, 5).cuda()
# model_norm1 = GConvRegressor(128, 5).cuda()
# model_norm2 = GConvRegressor(128, 5).cuda()

# model_skip2 = GConvRegressor(128, 5, skip=True, n_skip=2).cuda()
# model_skip3 = GConvRegressor(128, 5, skip=True, n_skip=3).cuda()

model_rnn_1 = GConvRegressor(128, 5, rnn_mode=True).cuda()
# model_rnn_3 = GConvRegressor(128, 5, rnn_mode=True, n_rnn_layer=3).cuda()
# model_rnn_3_dropout = GConvRegressor(128, 5, rnn_mode=True, n_rnn_layer=3, dropout=0.3).cuda()
        
#Dataset
train_smiles = smiles[:19000]
test_smiles = smiles[19000:20000]
train_logp = Y[:19000]
test_logp = Y[19000:20000]
train_dataset = MolDataset(train_smiles, train_logp, max_natoms)
test_dataset = MolDataset(test_smiles, test_logp, max_natoms)

train_dataset_norm1 = MolDataset(train_smiles, train_logp, max_natoms, norm=1)
test_dataset_norm1 = MolDataset(test_smiles, test_logp, max_natoms, norm=1)

train_dataset_norm2 = MolDataset(train_smiles, train_logp, max_natoms, norm=2)
test_dataset_norm2 = MolDataset(test_smiles, test_logp, max_natoms, norm=2)

#Dataloader
train_dataloader = DataLoader(train_dataset, batch_size=128, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=128, num_workers=0)

train_dataloader_norm1 = DataLoader(train_dataset_norm1, batch_size=128, num_workers=0)
test_dataloader_norm1 = DataLoader(test_dataset_norm1, batch_size=128, num_workers=0)

train_dataloader_norm2 = DataLoader(train_dataset_norm2, batch_size=128, num_workers=0)
test_dataloader_norm2 = DataLoader(test_dataset_norm2, batch_size=128, num_workers=0)

loss_fn = nn.MSELoss()


train_dataloaders = [train_dataloader_norm2]
test_dataloaders = [test_dataloader_norm2]
models = [model_rnn_1]
result = []

def train(model, train_dataloader):
  #model initialize
  for param in model.parameters():
      if param.dim() == 1:
          continue
          nn.init.constant(param, 0)
      else:
          nn.init.xavier_normal_(param)

  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  #optimizer = torch.optim.SGD(model.parameters(), lr=lr)

  loss_list = []
  st = time.time()

  for epoch in range(10):
    epoch_loss = []
    for i_batch, batch in enumerate(train_dataloader):
      x, y, A = \
        batch['X'].cuda().float(), batch['Y'].cuda().float(), batch['A'].cuda().float()
      pred = model(x, A).squeeze(-1)
      loss = loss_fn(pred, y)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      loss_list.append(loss.data.cpu().numpy())
      epoch_loss.append(loss.data.cpu().numpy())

    if True: print (epoch, np.mean(np.array(epoch_loss)))
  end = time.time()
  print ('Time:', end-st)
  return model, loss_list

for i in range(len(models)):
  print(i)
  result.append(train(models[i], train_dataloaders[i]))
  print('')


0
0 1.7397764
1 0.7418095
2 0.36624217
3 0.2700219
4 0.22563374
5 0.22531548
6 0.22045867
7 0.18192194
8 0.15056051
9 0.122955665
Time: 1143.0888187885284



In [ ]:
#Save model
fn = '/content/gdrive/My Drive/save.pt'
torch.save(model.state_dict(), fn)

In [ ]:
#Load model
model.load_state_dict(torch.load(fn))

In [ ]:
import matplotlib.pyplot as plt

for i in range(len(result)):
  plt.plot(result[i][1])
  plt.xlabel('Num iteration')
  plt.ylabel('Loss')


In [ ]:
#Test model
def loss_check(model, train_dataloader, test_dataloader):
  model.eval()
  with torch.no_grad():
    y_pred_train, y_pred_test = [], []
    loss_train, loss_test = [], []
    pred_train, pred_test = [], []
    true_train, true_test = [], []

    
    for batch in train_dataloader:
      x, y, A = \
        batch['X'].cuda().float(), batch['Y'].cuda().float(), batch['A'].cuda().float()
      pred = model(x, A).squeeze(-1)
      pred_train.append(pred.data.cpu().numpy())
      true_train.append(y.data.cpu().numpy())

      loss_train.append(loss_fn(y, pred).data.cpu().numpy())
    
    for batch in test_dataloader:
      x, y, A = \
        batch['X'].cuda().float(), batch['Y'].cuda().float(), batch['A'].cuda().float()
      pred = model(x, A).squeeze(-1)
      pred_test.append(pred.data.cpu().numpy())
      true_test.append(y.data.cpu().numpy())
      loss_test.append(loss_fn(y, pred).data.cpu().numpy())

  pred_train = np.concatenate(pred_train, -1)
  pred_test = np.concatenate(pred_test, -1)
  true_train = np.concatenate(true_train, -1)
  true_test = np.concatenate(true_test, -1)
      
  print ('Train loss:', np.mean(np.array(loss_train)))
  print ('Test loss:', np.mean(np.array(loss_test)))
  print('')

for i in range(len(models)):
  loss_check(result[i][0], train_dataloaders[i], test_dataloaders[i])

Train loss: 0.14034033
Test loss: 0.12209468



In [ ]:
.
plt.scatter(true_train, pred_train, s=1)
plt.scatter(true_test, pred_test, s=1)

plt.plot([-8,12], [-8,12])
plt.xlabel('True')
plt.ylabel('Pred')


NameError: ignored